In [6]:
import logging
import os
import sys
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np

from models.resnet import resnet18 

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()



MONAI version: 1.5.0
Numpy version: 2.0.1
Pytorch version: 2.5.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: d388d1c6fec8cb3a0eebee5b5a0b9776ca59ca83
MONAI __file__: /home/<username>/miniconda3/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.15.1
Pillow version: 11.1.0
Tensorboard version: 2.18.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.66.5
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNO

https://share.weiyun.com/55sZyIx

In [7]:
# Dimensiones de tus volúmenes de MRI usamos 256x256x256 ya que es un tamaño común para imágenes de resonancia magnética cerebral en freesurfer
input_D = 256
input_H = 256
input_W = 256

# Número de canales (1 para norm.mgz)
input_C = 1 
model = resnet18(sample_input_D=input_D,
                 sample_input_H=input_H,
                 sample_input_W=input_W,
                 num_seg_classes=1)  # ATENTO A ESTO PORQUE ES IMPORTANTE PARA LA CLASIFICACION

In [8]:
# Ruta completa a tu archivo de pesos pre-entrenados de ResNet-18
pretrained_weights_path = 'path/to/your/resnet_18.pth' # ¡Actualiza esta ruta!

try:
    state_dict = torch.load(pretrained_weights_path)
    model.load_state_dict(state_dict, strict=False)
    print(f"Pesos de ResNet-18 cargados exitosamente desde {pretrained_weights_path}")

except FileNotFoundError:
    print(f"Error: El archivo de pesos no se encontró en {pretrained_weights_path}")
    print("Asegúrate de que la ruta sea correcta y el archivo exista.")
except Exception as e:
    print(f"Ocurrió un error al cargar los pesos: {e}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modelo movido a: {device}")

/tmp/ipykernel_32364/416200560.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_weights_path)


Error: El archivo de pesos no se encontró en path/to/your/resnet_18.pth
Asegúrate de que la ruta sea correcta y el archivo exista.
Modelo movido a: cuda


In [ ]:

num_ftrs = model.fc.in_features
your_num_classes = 2 # Por ejemplo, para clasificación binaria, tiene o no tiene alzheimer

model.fc = nn.Linear(num_ftrs, your_num_classes)
print(f"Capa de clasificación final adaptada a {your_num_classes} clases.")

# Mueve la nueva capa (que no tiene pesos pre-entrenados) a la GPU también
model.to(device)